In [1]:
import jwt
import requests
import os

PORT = os.environ.get("PORT", 5000)

# If the environment if it was on production or on testing mode
init_url = 'https://test.zaincash.iq/transaction/init'
request_url = 'https://test.zaincash.iq/transaction/pay?id='

if os.environ.get("PRODUCTION") == "true":
    init_url = 'https://api.zaincash.iq/transaction/init'
    request_url = 'https://api.zaincash.iq/transaction/pay?id='

# Set the serviceType (Any text you like such as your website name)
service_type = "book"

# After a successful or failed order, the user will redirect to this URL
redirect_url = 'https://example.com/redirect'

"""
Notes about redirectionUrl:
In this URL, the API will add a new parameter (token) to its end like:
https://example.com/redirect?token=XXXXXXXXXXXXXX
"""

# Handling the payment request
def handle_payment_request():
    # Set the amount to 250 if there is no amount in the request (For testing)
    # It has to be more than 250 IQD
    amount = 250

    # Set an order id (This usually should be the order id in your sys DB)
    order_id = "YOUR_ORDER_ID_FROM_YOUR_DB"

    # Set the token expire time
    time = int(datetime.now().timestamp())

    # Building the transaction data to be encoded in a JWT token
    data = {
        'amount': amount,
        'serviceType': service_type,
        'msisdn': os.environ.get("MSISDN"),
        'orderId': order_id,
        'redirectUrl': redirect_url,
        'iat': time,
        'exp': time + 60 * 60 * 4
    }

    # Encoding the data
    token = jwt.encode(data, os.environ.get("SECRET"), algorithm="HS256")

    # Preparing the payment data to be sent to ZC API
    post_data = {
        'token': token,
        'merchantId': os.environ.get("MERCHANTID"),
        'lang': os.environ.get("LANG")
    }

    # Request options
    request_options = {
        'uri': init_url,
        'json': post_data,
        'method': 'POST',
        'headers': {
            'Content-Type': 'application/json'
        }
    }

    # Initializing a ZC order by sending a request with the tokens
    response = requests.post(**request_options)

    # Getting the operation id
    operation_id = response.json().get("id")

    # Redirect the user to ZC payment Page
    redirect_url = request_url + operation_id
    print(f"Redirecting to: {redirect_url}")


# Handling the redirect
def handle_redirect(token):
    if token:
        try:
            decoded = jwt.decode(token, os.environ.get("SECRET"), algorithms=["HS256"])
        except jwt.exceptions.DecodeError:
            # Handle decoding error
            pass
        else:
            if decoded.get("status") == 'success':
                # Do whatever you like
                pass
            else:
                # Do other things
                pass


# Running the script from the terminal
if __name__ == "__main__":
    command = input("Enter command (request or redirect): ")
    
    if command == "request":
        handle_payment_request()
    elif command == "redirect":
        token = input("Enter token: ")
        handle_redirect(token)
    else:
        print("Invalid command")


Invalid command


In [5]:
import requests
import jwt
from time import time

# # Import credentials
# from credentials import msisdn, secret, merchantid, production_cred

msisdn = 9647835077893
secret = '$2y$10$hBbAZo2GfSSvyqAyV2SaqOfYewgYpfR1O19gIh4SqyGWdmySZYPuS'
merchantid = '5ffacf6612b5777c6d44266f'
production_cred = 0

# Request Details
id = '6488999a3e39a300e027cece'

# Building data
data = {
    'id': id,
    'msisdn': msisdn,
    'iat': time(),
    'exp': time() + 60 * 60 * 4
}

# Encoding Token
newtoken = jwt.encode(data, secret, algorithm='HS256')

# Check if test or production mode
if production_cred:
    rUrl = 'https://api.zaincash.iq/transaction/get'
else:
    rUrl = 'https://test.zaincash.iq/transaction/get'

# POST data to ZainCash API
data_to_post = {
    'token': newtoken,
    'merchantId': merchantid
}
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}
response = requests.post(rUrl, data=data_to_post, headers=headers)

print(response.text)


{"to":{"name":"Karrar","msisdn":"9647835077893","currency":"IQD","deleted":false,"pay_by_reference":"1","createdAt":"2021-01-10T09:56:54.180Z","updatedAt":"2021-12-22T13:01:02.531Z","id":"5ffacf6612b5777c6d44266f"},"source":"web","type":"MERCHANT_PAYMENT","amount":"250","serviceType":"A book","lang":"en","orderId":"Bill_1234567890","currencyConversion":{},"referenceNumber":"WTZYF6","redirectUrl":"http://localhost/redirect.php","credit":false,"status":"failed","reversed":false,"createdAt":"2023-06-13T16:30:18.367Z","updatedAt":"2023-06-13T16:34:17.154Z","sofOwnerId":18482,"traveldiscount":"250","due":"The requester is not allowed to perform this operation","from":"9647802999569","id":"6488999a3e39a300e027cece"}


In [ ]:
import requests
import json
import jwt
import time
from pprint import pprint 

# Set your Zain Cash API credentials
merchant_id = '5ffacf6612b5777c6d44266f'
secret_key = '$2y$10$hBbAZo2GfSSvyqAyV2SaqOfYewgYpfR1O19gIh4SqyGWdmySZYPuS'
api_url = 'https://test.zaincash.iq/transaction/init'
redirect_url = 'http://localhost/redirect.php'

# Set the order details
amount = 250
service_type = 'A book'
order_id = 'Bill_1234567890'

# Build the data
data = {
    'amount': amount,
    'serviceType': service_type,
    'msisdn': '9647835077893',
    'orderId': order_id,
    'redirectUrl': redirect_url,
    'iat': int(time.time()),
    'exp': int(time.time()) + 60 * 60 * 4
}

# Encode the data as a JWT token
token = jwt.encode(data, secret_key, algorithm='HS256')

# Prepare the POST data
post_data = {
    'token': token,
    'merchantId': merchant_id,
    'lang': 'en'
}

# Send the request to ZainCash API
response = requests.post(api_url, data=post_data)

# Parsing response
response_data = json.loads(response.text)
transaction_id = response_data['id']

# Construct the payment URL
payment_url = f'https://test.zaincash.iq/transaction/pay?id={transaction_id}'
print(f'Redirect the user to: {payment_url}')
